# DeepLabCut + XROMM Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath*, Mathis* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf



XROMMTools for DeepLabCut written by J.D. Laurence-Chasen

This notebook walks you through the XMALab/DeepLabCut workflow, from project creation to analysis.

At this stage, you should have tracked data in XMALab and exported the 2D points for trials you wish to be included in the initial training dataset. Each trial should have its own subfolder, containing **only** the images (image stack or video files) and the 2D points CSV. 

In [1]:
# Import packages. Run this every time!
import deeplabcut
from deeplabcut.utils import xrommtools

## Create a new project

The first step is to create a DeepLabCut project (network). Whether or not you create new projects for each day of data collection depends in large part on the variation in the dataset and number of markers. You should always create new projects for different individuals.

N.B. If you are creating a new project for a different day of data collection on the same animal (or an animal with a similar marker scheme), it may be easier to manually copy an existing project and delete the folder contents to minimize config file editing.

In [ ]:
# RUN THIS SECTION ONLY ONCE, when you first create the project. 
# If a symbolic link error occurs, make sure you've run Anaconda Navigator "as Administrator"

task='PigFeeding' # Enter the name of your experiment Task
experimenter='JD' # Enter the name of the experimenter
video=['C:/Users/jdlc7/PigDLC/Pig_20191203.avi'] # IMPORTANT: This video is a 'dummy' video. Just point the function to something.
working_directory = 'C:/Users/jdlc7/Pig_20191203' # where the project will be created; use forward slashes on Windows

path_config_file=deeplabcut.create_new_project(task,experimenter,video,working_directory,copy_videos=False)

## Now edit the newly created configuration file to match your project. 

See DLC methods paper for the significance of the different terms.

Make sure the body parts correspond **EXACTLY** to your point names/order in XMALab (this is why consistent naming is important)

In [ ]:
# Set working directory paths if you haven't yet. This is is for when you return to this notebook (so you don't re-run the cell above).

path_config_file = 'C:/Users/jdlc7/PigDLC/PigFeeding-JD-2019-12-03/config.yaml'

## Convert XMALab 2D points to DLC format

Now we extract and convert the 2D points file/s and frames from the videos. The function xma_to_dlc will populate the project's labeled-data folder with the extracted data, such that it's ready for DeepLabCut's native create_training_dataset function.

**Be sure** that dataset_name exactly matches the dummy video under "video sets" in the config file (but without the .avi).
So if your dummy video in the config file is '.../Pig_20191203.avi', then dataset name must be 'Pig_20191203'.

**Also**, be sure your 2D points files don't contain any leading or trailing spaces.
You can get rid of those using Excel's trim() function.

In [ ]:
experimenter = 'JD'
data_path = 'C:/Users/jdlc7/Desktop/Pig_20191203/DLC' # where are the trial folders (with 2D points/trial images)
dataset_name = 'Pig_20191203'
nframes = 400 # how many frames do you want the training dataset to comprise? Make sure it's equal to or less than the total numbed of tracked frames

xrommtools.xma_to_dlc(path_config_file,data_path,dataset_name,experimenter,nframes)

## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typically, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Start training:

This function trains the network for a specific shuffle of the training dataset. This will take a long time. You can stop training manually by hitting stop. It will look like an error has occured...it has not. 

In [ ]:
deeplabcut.train_network(path_config_file) # there are many other parameters you can set here

## Evaluating Network

This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**. The values here can give you an indication of performance/generalizability, but are less informative than reprojection error and other XMALab metrics. It's recommended you use this to assess whether something went really wrong (i.e. errors higher than 2-3 pixels.), but importing predicted points into XMALab is the best way to assess performance.

In [ ]:
deeplabcut.evaluate_network(path_config_file)

## Analyzing videos

We call the existing DLC function analyze_videos to predict points for new trials and convert the output to XMALab format. The results are stored in hd5 file and CSV file in each trial's specific iteration subfolder.

The base function analyze_videos has many possible arguments. To alter this, you will need to edit the xrommtools script manually. Its default form should be sufficient in most cases.


**Note, this has been tested only on .avi files**. If you have jpg stacks (which is fine for creating or augmenting the training dataset), we recommend you convert to .avi if you want to predict points here. 

In [ ]:
new_data_path = 'C:/Users/jdlc7/Desktop/Pig_20191203/Trials' # where are the trials you wish to analyze? 
# IMPORTANT: new_data_path must only contain trial folders, which in turn must only contain video files.
# The function automatically reads camera 1 and camera 2 from the file name, thus any other files/folders will cause an error.

iteration = 0 # what iteration of the network do you want to analyze the videos?
#Start at 0, and if you augment and retrain, update this as well as the config file

xrommtools.analyze_xromm_videos(path_config_file, new_data_path, iteration)

## Augment the training dataset with corrected frames [optional step]

If performance isn't adaqute (i.e. high reprojection errors), you can correct frames in XMALab and add them to the training dataset, and re-train iteratively. 

Following the correction of frames in XMALab and the re-exporting of 2D points, you can augment the training dataset and retrain the networks. 

**Make sure** the newley exported 2D points files have some form of 'corrected' in their file names, and that they are located in the itX subfolder of each trial.

Correct formatting of the frames file is essential, so refer to the instructions to make sure you have it right.

In [ ]:
frames = 'C:/Users/jdlc7/PigDLC/frameindex.csv' # column 1 should be trialnames, and columns 2+ should be frame numbers

new_data_path = 'C:/Users/jdlc7/Desktop/Pig_20191203/Trials' # where are trial folders with new videos to analyze?
#This should be the same as new_data_path for analyzing videos

iteration = 0 # What iteration subfolder should the function look for the corrected 2D points files in?

xrommtools.add_frames(path_config_file, new_data_path, iteration, frames)

## Create a new iteration of training dataset

Once you've added frames to the training dataset using the add_frames function, you can now use the native function once again to create a new iteration of the training dataset.

**It is essential** that at this point you update the iteration in the config file, i.e. 0 --> 1 after the first augmentation.

After you re-create the training dataset, you can re-run train_network and repeat all the following steps until performance is acceptable (or it plateaus).
If you want to resume training with the existing weights, rather than starting from scractch, be sure to edit init_weights in the pose_config.yaml file.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [ ]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)